In [583]:
import math
import numpy as np
import numpy.linalg as la
from importlib import reload
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import beta, norm, gamma, kstest, alpha, t, rice
a4_dims = (11.7, 8.27)

In [450]:
x = np.array([1,0,0])
la.norm(x)

1.0

In [451]:
import src.analysis.utils as ana_utils
reload(ana_utils)
reload(src.experiment.utils)

<module 'src.experiment.utils' from '/home/garro/Projects/masterarbeit/src/experiment/utils.py'>

In [648]:
xs = ana_utils.read_improvement_probs('/media/sf_VBoxshare/Experimente/output/static_2/probsat.db', 7)
distr = np.array([p for _, p in xs])
print(distr)
d = ana_utils.build_transition_matrix(distr[0])
d

[[0.         0.69856964 0.69919458 ... 0.91450699 1.         1.        ]
 [0.         0.73030428 0.74472444 ... 0.93427119 1.         1.        ]
 [0.         0.015625   0.78582533 ... 0.92115614 0.95924642 1.        ]
 ...
 [0.         0.6873695  0.74819285 ... 0.90149953 0.94239253 1.        ]
 [0.         0.80464217 0.69186826 ... 0.9243685  0.94247364 1.        ]
 [0.         0.54961857 0.68798094 ... 0.94150706 0.9214649  1.        ]]


array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.01430363e-01, 0.00000000e+00, 6.98569637e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 3.00805418e-01, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 9.14506991e-01, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        2.22044605e-16, 0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [629]:
xs = ana_utils.get_tms_entropy('/media/sf_VBoxshare/Experimente/output/static_2/gsat.db', 1, eps=2**(-20))

In [630]:
plot_data = []
all_data = []
for _, h in xs:
    if h > 0.85:
        plot_data.append(h)
    all_data.append(h)

KeyboardInterrupt: 

In [ ]:
fig, ax = plt.subplots(figsize=a4_dims)
sns.distplot(plot_data, bins=20, norm_hist=True, fit=norm)
xs = np.arange(0.0,1,0.0001)
theta_beta = beta.fit(plot_data)
#ax.plot(xs, beta.pdf(xs, *theta_beta)) #3.28, 1.714, loc=0.9652, scale=0.0276))

In [509]:
theta_beta = beta.fit(plot_data)
theta_norm = norm.fit(plot_data)
theta_gamma = gamma.fit(plot_data)

In [510]:
theta_beta, theta_norm, theta_gamma

((3.513385113452088,
  2.108392349835383,
  0.9694237638605796,
  0.023439063459984533),
 (0.9840433571586524, 0.004332532026588534),
 (346.5061639585356, 0.9028080566116929, 0.0002344766370200313))

In [511]:
ana_utils.log_likelihood(all_data, lambda x: beta.pdf(x,*theta_beta))

57.22510131845356

In [512]:
ana_utils.log_likelihood(all_data, lambda x: norm.pdf(x,*theta_norm))

58.31138814480122

In [513]:
ana_utils.log_likelihood(all_data, lambda x: gamma.pdf(x,*theta_gamma))

58.060825234497585

In [514]:
kstest(all_data, lambda x: beta.cdf(x, *theta_beta))

KstestResult(statistic=0.10459381259745493, pvalue=0.20905216268298)

In [631]:
distr = beta

for i in range(1,11):
    xs = ana_utils.get_tms_entropy('/media/sf_VBoxshare/Experimente/output/static_2/gsat.db', i, eps=2**(-10))
    clean_data = []
    all_data = []
    for _, h in xs:
        if h > 0.85:
            clean_data.append(h)
        all_data.append(h)
        
    theta = distr.fit(clean_data)
    res = kstest(all_data, lambda x: distr.cdf(x,*theta))
    print(res.statistic)
    if res.statistic >= 0.1358:
        print('Fail')
    else:
        print('Passed')
    

0.10459381259745493
Passed
0.07856727615277032
Passed
0.12152880911486931
Passed
0.10306059115815691
Passed
0.1206863482278211
Passed
0.09854437408393002
Passed
0.11656194167509143
Passed
0.1247484306421133
Passed
0.09238260740118835
Passed
0.12826464567958093
Passed


In [519]:
1.358/math.sqrt(100)

0.1358